In [ ]:
from scipy.spatial import distance
import mediapipe as mp
from pygame import mixer
import cv2
import numpy as np  # Make sure numpy is imported

# Initialize pygame for sound alerts
mixer.init()
mixer.music.load("music.wav")

def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Threshold values
thresh = 0.25
frame_check = 20

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

cap = cv2.VideoCapture(0)
flag = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (450, 300))
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Get landmark points for left and right eyes
            left_eye = [
                (face_landmarks.landmark[362].x, face_landmarks.landmark[362].y),
                (face_landmarks.landmark[385].x, face_landmarks.landmark[385].y),
                (face_landmarks.landmark[387].x, face_landmarks.landmark[387].y),
                (face_landmarks.landmark[263].x, face_landmarks.landmark[263].y),
                (face_landmarks.landmark[373].x, face_landmarks.landmark[373].y),
                (face_landmarks.landmark[380].x, face_landmarks.landmark[380].y),
            ]

            right_eye = [
                (face_landmarks.landmark[33].x, face_landmarks.landmark[33].y),
                (face_landmarks.landmark[160].x, face_landmarks.landmark[160].y),
                (face_landmarks.landmark[158].x, face_landmarks.landmark[158].y),
                (face_landmarks.landmark[133].x, face_landmarks.landmark[133].y),
                (face_landmarks.landmark[153].x, face_landmarks.landmark[153].y),
                (face_landmarks.landmark[144].x, face_landmarks.landmark[144].y),
            ]

            # Convert normalized landmarks to pixel coordinates
            left_eye = [(int(x * frame.shape[1]), int(y * frame.shape[0])) for (x, y) in left_eye]
            right_eye = [(int(x * frame.shape[1]), int(y * frame.shape[0])) for (x, y) in right_eye]

            leftEAR = eye_aspect_ratio(left_eye)
            rightEAR = eye_aspect_ratio(right_eye)
            ear = (leftEAR + rightEAR) / 2.0

            # Draw eye contours without using UMat
            cv2.polylines(frame, [cv2.convexHull(np.array(left_eye))], True, (0, 255, 0), 1)
            cv2.polylines(frame, [cv2.convexHull(np.array(right_eye))], True, (0, 255, 0), 1)

            if ear < thresh:
                flag += 1
                print(flag)
                if flag >= frame_check:
                    cv2.putText(frame, "*****ALERT!*****", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, "*****ALERT!*****", (10, 260),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    mixer.music.play()
            else:
                flag = 0

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()